### Read data from file

In [ ]:
import pickle
from pathlib import Path

root_module = Path.cwd()
ext = ".pkl"
file_name = "dmc_quantile_optimism"
load_dir = root_module.parent.joinpath("data/")
file_dir = load_dir.joinpath(file_name + ext)
raw_data = pickle.load(open(file_dir, "rb"))

### Process raw results

In [ ]:
from dist_mbrl.utils.process_results import get_mean_median_returns, process_raw_results

# Post-process raw return data to get mean and standard error
metric_data = process_raw_results(raw_data)

# Aggregate metric_data across environments for global statistics
aggregated_data = get_mean_median_returns(metric_data)

# Retrieve relevant variables from data
env_names = metric_data["env_names"]
steps = metric_data["steps"]
mean_returns = metric_data["mean_returns"]
ci_returns = metric_data["ci_returns"]

### Plotting

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

from dist_mbrl.utils.plot import (
    JMLR_PARAMS,
    handle_1D_axes_and_legend,
    plot_with_symmetric_intervals,
)

plt.rcParams.update(JMLR_PARAMS)

envs_to_plot = [
    "cheetah-run",
    "walker-run",
    "reacher-hard",
    "finger-spin",
]

# Define grid of plots
ncols = 4
nrows = 1
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(6.8, 1.2),
    gridspec_kw={"wspace": 0.45, "hspace": 0.50},
)

# Reference axes by environment name
ax_dict = {}
for idx, ax in zip(envs_to_plot, axes.flatten()):
    ax_dict[idx] = ax

colors = {}
cmap = plt.get_cmap("tab10")
for i, type in enumerate(mean_returns[env_names[0]].keys()):
    colors[type] = cmap(i)


# Custom processing of labels
def custom_process_label(params):
    quantile = params[0]
    agent_type = params[1]
    if quantile == "nan":
        return rf"\texttt{{{agent_type}}}"
    else:
        quantile = int(quantile) / 100.0
        return rf"\texttt{{{agent_type}-{quantile}}}"


ep_length = 1000
for env_name in envs_to_plot:
    for idx in mean_returns[env_name].keys():
        plot_with_symmetric_intervals(
            ax=ax_dict[env_name],
            x=steps[env_name][idx] // ep_length,
            y=mean_returns[env_name][idx],
            yerr=ci_returns[env_name][idx],
            label=custom_process_label(idx[1:]),
            title=env_name,
            color=colors[idx],
        )

handle_1D_axes_and_legend(axes=axes, legend_ncol=4, legend_offset=(2.6, -0.8))

### Save figures


In [ ]:
fig_dir = root_module.parent.joinpath("figures/dmc_quantile_optimism.pdf")
fig.savefig(fig_dir, bbox_inches="tight", transparent=False)

# License

>Copyright (c) 2024 Robert Bosch GmbH
>
>This program is free software: you can redistribute it and/or modify <br>
>it under the terms of the GNU Affero General Public License as published<br>
>by the Free Software Foundation, either version 3 of the License, or<br>
>(at your option) any later version.<br>
>
>This program is distributed in the hope that it will be useful,<br>
>but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
>GNU Affero General Public License for more details.<br>
>
>You should have received a copy of the GNU Affero General Public License<br>
>along with this program.  If not, see <https://www.gnu.org/licenses/>.